In [1]:
import pandas as pd
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('germany_cars.csv')
df.drop(columns=['model'],inplace = True)

In [4]:
df.title_status = np.where(df.title_status=="parts only","salvage",df.title_status)

In [5]:
lables = ['missing','lien','salvage','rebuilt','clean']
nominal = np.arange(0,5,step = 1)
df.title_status = df.title_status.replace(lables,nominal)
df.title_status = df.title_status.astype(np.int32)
df.transmission = np.where(df.transmission=="automatic",1,0)

In [6]:
df = pd.get_dummies(df,prefix="",prefix_sep="")

In [7]:
df.odometer = np.sqrt(df.odometer)
df.price = np.sqrt(df.price)

In [8]:
X = df.drop(columns=['price'])
y = df.price

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.4,random_state=42)

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
lm = LinearRegression().fit(X_train,y_train)

In [13]:
lm.score(X_train,y_train)

0.8566967820141942

In [14]:
lm.score(X_test,y_test)

0.8517022404305815

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
def RMSE(y_true,y_pred):
    
    y_true = y_true**2
    y_pred = y_pred**2
    
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [17]:
pred_train = lm.predict(X_train)
pred_test = lm.predict(X_test)

In [19]:
RMSE(y_train,pred_train)

5223.227158645108

In [20]:
RMSE(y_test,pred_test)

6101.204803682678

In [21]:
from sklearn.model_selection import cross_val_score

In [22]:
cross_val_score(lm,X_test,y_test,cv = 5)

array([0.85069728, 0.85790412, 0.84511156, 0.86022043, 0.84383952])

In [23]:
columns = X.columns

In [24]:
def predict(manufacturer,body,year,odometer,cyl,engine,fuel,transmission,hp,title_status):

    zeros = np.zeros(len(columns))
    title_status_dict = {'missing':0,'lien':1,'salvage':2,'rebuilt':3,'clean':4}
    zeros[0] = year
    zeros[1] = cyl
    zeros[2] = np.sqrt(odometer)
    zeros[3] = title_status_dict[title_status]
    zeros[4] = np.where(transmission=="automatic",1,0)
    zeros[5] = engine
    zeros[6] = hp


    manufacturer_idx = np.where(manufacturer == columns)[0][0]
    type_body_idx = np.where(body == columns)[0][0]
    fuel_idx = np.where(fuel == columns)[0][0]




    if manufacturer_idx >= 0:
        zeros[manufacturer_idx] = 1
        
   


    if fuel_idx >=0:
        zeros[fuel_idx] = 1


    if type_body_idx>=0:
        zeros[type_body_idx] = 1



    zeros = np.asarray([zeros])

    pred = lm.predict(zeros)


    return pred**2

In [25]:
predict('bmw','sedan',2023,1770,6,3.0,'gas','automatic',382,'clean')

array([56220.58728116])

In [38]:
predict('volkswagen','hatchback',2023,1516,4,2.0,'gas','manual',241,'clean')

array([46623.32041572])